In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [3]:
df=pd.read_csv("data.csv",header=None)

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,...,-28.0,61.0,4.0,8.0,5.0,4.0,-7.0,-59.0,16.0,0
1,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,...,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
2,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,...,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
3,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,...,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
4,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,...,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,-12.0,0.0,0.0,-7.0,-4.0,-3.0,-1.0,-5.0,4.0,-1.0,...,12.0,-3.0,0.0,2.0,-1.0,-1.0,4.0,-30.0,-20.0,0
2906,-9.0,-1.0,-2.0,1.0,-13.0,-14.0,-59.0,-4.0,-9.0,0.0,...,20.0,-4.0,0.0,0.0,0.0,-21.0,-10.0,-14.0,-29.0,0
2907,3.0,-2.0,-2.0,4.0,18.0,-5.0,-31.0,7.0,-3.0,-3.0,...,34.0,-7.0,4.0,4.0,2.0,1.0,4.0,-55.0,-4.0,0
2908,-2.0,-2.0,-2.0,1.0,-3.0,-17.0,-33.0,-13.0,3.0,2.0,...,-4.0,0.0,3.0,0.0,5.0,9.0,8.0,-13.0,11.0,0


In [5]:
dataset_train = df.iloc[np.random.permutation(len(df))]
dataset_train.reset_index(drop=True)


,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,-12.0,-5.0,-1.0,4.0,-16.0,-17.0,-69.0,-16.0,-12.0,-3.0,...,1.0,-36.0,-10.0,-12.0,-16.0,-12.0,-47.0,6.0,-30.0,0
1,-21.0,4.0,2.0,22.0,-9.0,6.0,-3.0,1.0,-11.0,-2.0,...,-10.0,-8.0,0.0,9.0,-13.0,-3.0,1.0,14.0,0.0,0
2,16.0,5.0,-5.0,-10.0,5.0,29.0,90.0,49.0,-6.0,2.0,...,6.0,-35.0,6.0,5.0,13.0,5.0,15.0,-52.0,27.0,0
3,-3.0,3.0,-2.0,2.0,-8.0,8.0,-4.0,-16.0,10.0,3.0,...,-38.0,4.0,5.0,-1.0,-8.0,-4.0,21.0,61.0,8.0,0
4,-5.0,-10.0,-13.0,2.0,18.0,8.0,47.0,7.0,3.0,-3.0,...,-9.0,-8.0,6.0,-4.0,-7.0,-13.0,-25.0,57.0,16.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2905,1.0,-4.0,-7.0,-9.0,2.0,11.0,-32.0,1.0,0.0,2.0,...,-23.0,6.0,-5.0,-7.0,-9.0,-5.0,-9.0,-38.0,23.0,0
2906,-3.0,14.0,3.0,2.0,13.0,26.0,-14.0,7.0,12.0,16.0,...,-34.0,11.0,-11.0,-6.0,4.0,-4.0,16.0,127.0,24.0,0
2907,-50.0,14.0,-11.0,4.0,-8.0,2.0,-13.0,6.0,87.0,-24.0,...,22.0,-28.0,-16.0,-1.0,-2.0,13.0,30.0,-14.0,-11.0,0
2908,-26.0,-13.0,-7.0,-6.0,35.0,-6.0,23.0,17.0,0.0,8.0,...,5.0,3.0,6.0,-5.0,0.0,-11.0,-5.0,-11.0,-17.0,0


In [6]:
dataset_train.reset_index(drop=True)
X_train = []
y_train = []

In [7]:
for i in range(0, dataset_train.shape[0]):
    row = np.array(dataset_train.iloc[i:1+i, 0:64].values)
    X_train.append(np.reshape(row, (64, 1)))
    y_train.append(np.array(dataset_train.iloc[i:1+i, -1:])[0][0])

In [8]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
X_train = X_train.reshape(X_train.shape[0]*X_train.shape[1], 1)
X_train = sc.fit_transform(X_train)

In [10]:
X_train = X_train.reshape((-1, 8, 8))

In [11]:
y_train = np.eye(np.max(y_train) + 1)[y_train]

In [13]:
X_train.shape

(2910, 8, 8)

In [14]:
y_train.shape

(2910, 1)

In [15]:
X_test = X_train[7700:]
y_test = y_train[7700:]

In [17]:
X_train.shape

(2910, 8, 8)

In [31]:
model = Sequential()
tf.config.run_functions_eagerly(True)
model.add(Dense(10, input_dim=5, activation='relu'))
model.add(Dense(4, activation='softmax'))
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
y_one_hot = tf.one_hot(X_train, depth=4)

# Define the new loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Compile the model with the new loss function
model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])
model.fit(X_test, y_test, epochs=10, batch_size=32)
model.compile(optimizer='adam', loss='binary_crossentropy', run_eagerly=True)

Epoch 1/10


ValueError: Unexpected result of `train_function` (Empty logs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [23]:
classifier = Sequential()

classifier.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 8)))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50, return_sequences = True))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50, return_sequences = True))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 64))
classifier.add(Dense(units = 128))

classifier.add(Dense(units = 4, activation="softmax"))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy')

classifier.fit(X_train, y_train, epochs = 100, batch_size = 32)
# model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100


ValueError: in user code:

    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-packages/keras/engine/training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-packages/keras/engine/training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-packages/keras/engine/training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-packages/keras/engine/training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-packages/keras/engine/training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "/home/kishore/miniconda3/envs/ml310/lib/python3.10/site-p